In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from bsuite.baselines.base import Agent
import dm_env
from helx.rl import baselines
import gym
from gym_minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper
from bsuite.utils.gym_wrapper import DMEnvFromGym
import wandb
import logging
import helx

In [9]:
def make(name):
    env = gym.make(name)
    env = RGBImgPartialObsWrapper(env)  # Get pixel observations
    env = ImgObsWrapper(env)  # Get rid of the 'mission' field
    env = DMEnvFromGym(env)  #  Convert to dm_env.Environment
    return env

In [6]:
env = make("MiniGrid-Empty-6x6-v0")
module = baselines.dqn
hparams = module.HParams(replay_memory_size=1000, replay_start=10, batch_size=4)
agent = module.Dqn(env.observation_spec(), env.action_spec(), hparams)

NameError: name 'gym' is not defined

In [ ]:
a = helx.rl.base.run(agent, env, 10000)

In [13]:
from helx.rl.buffer import Transition
import jax.numpy as jnp

env = make("MiniGrid-Empty-6x6-v0")
transitions = [
    Transition(
        x_0=jnp.zeros(env.observation_spec().shape),
        a_0=jnp.zeros((1,)),
        r_0=jnp.zeros((1,)),
        x_1=jnp.zeros(env.observation_spec().shape),
    )
] * 10


def tree_vmap(f, lst):
    stacked = jax.tree_map(lambda args: jnp.stack(args), lst)
    out_stacked = jax.vmap(f)(stacked)
    _, outer_treedef = jax.tree_flatten([None] * len(lst))
    _, inner_treedef = jax.tree_flatten(out_stacked)
    out_unstacked_transposed = jax.tree_map(list, out_stacked)
    out_unstacked = jax.tree_transpose(
        outer_treedef, inner_treedef, out_unstacked_transposed
    )
    return out_unstacked


def batch_transitions(transitions):
    

In [32]:
import jax
jax.tree_util.tree_reduce(jnp.stack, list(map(lambda x: tuple(x), transitions)))

TypeError: only integer scalar arrays can be converted to a scalar index